In [1]:
import cv2
import numpy as np

# 读取两张图像
img1 = cv2.imread("D:/test_pic/3_1.png")
img2 = cv2.imread("D:/test_pic/3_2.png")

# 提取ORB特征点和描述符
orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

# 特征点匹配
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = matcher.match(des1, des2)

# 筛选出最佳匹配
matches = sorted(matches, key=lambda x: x.distance)
good_matches = matches[:64]

# 获取匹配对应的特征点坐标
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

# 求解基础矩阵
F, mask = cv2.findFundamentalMat(src_pts, dst_pts, cv2.FM_RANSAC)
essential_matrix, _ = cv2.findEssentialMat(src_pts, dst_pts, focal=1, pp=(0,0), method=cv2.RANSAC, prob=0.999, threshold=1.0)

In [2]:
# 恢复相机姿态
retval, R, t, mask = cv2.recoverPose(essential_matrix, src_pts, dst_pts)

# 构建相机投影矩阵(P)和反投影矩阵(Pinv)
P_left = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0]])
P_right = np.hstack((R, t))
print(P_right.shape)
homo_row = np.array([0, 0, 0, 1]).reshape((1, 4))
P_right = np.vstack((P_right, homo_row))
P_right[:, 3] = [0, 0, 0, 1]
Pinv = np.linalg.pinv(P_right)

(3, 4)


In [3]:
# 三角化重建
points_4d = cv2.triangulatePoints(P_left, P_right, src_pts, dst_pts)
points_3d = cv2.convertPointsFromHomogeneous(points_4d.T)

# 显示结果
for i in range(points_3d.shape[0]):
    print("Point ", i, " : ", points_3d[i][0])

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\triangulate.cpp:82: error: (-209:Sizes of input arguments do not match) Size of projection matrices must be 3x4 in function 'icvTriangulatePoints'


In [ ]:
import cv2
import numpy as np

# 加载图像并找到特征点
img_left = cv2.imread("D:/test_pic/3_1.png")
img_right = cv2.imread("D:/test_pic/3_2.png")

sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img_left, None)
kp2, des2 = sift.detectAndCompute(img_right, None)

# 使用FLANN匹配器进行特征点匹配
matcher = cv2.FlannBasedMatcher()
matches = matcher.match(des1, des2)

# 根据匹配结果获取对应的特征点坐标
src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1, 1, 2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1, 1, 2)

# 计算基础矩阵
F, mask = cv2.findFundamentalMat(src_pts, dst_pts, cv2.FM_RANSAC)

# 根据基础矩阵计算本质矩阵
K = np.array([[1, 0, img_left.shape[1]/2], [0, 1, img_left.shape[0]/2], [0, 0, 1]])
E = np.matmul(np.matmul(K.T, F), K)
R1, R2, t = cv2.decomposeEssentialMat(E)

# 构造相机投影矩阵
P_left = np.hstack((np.eye(3), np.zeros((3,1))))
P_right = np.hstack((R2, t))

# 构造匹配点对，进行三角化
points_4d = cv2.triangulatePoints(P_left, P_right, src_pts, dst_pts)
points_3d = (points_4d / points_4d[3,:])[:3,:].T

# 显示结果
print(points_3d)
cv2.imshow("Left Image", img_left)
cv2.imshow("Right Image", img_right)
cv2.waitKey(0)

[[-8.2504861e-02 -3.5726022e-02 -7.2445441e-04]
 [-1.4046472e-02 -1.0102340e-02 -4.8160055e-05]
 [-1.1889264e-01 -5.8622055e-02 -8.1439130e-04]
 ...
 [-8.7971318e-01 -3.9275861e-01 -2.8914839e-03]
 [-4.1825688e-01 -1.9074553e-01 -1.3811325e-03]
 [-9.0516090e-01 -4.0388402e-01 -3.1076462e-03]]
